# 0. 개요

- 모든 시간대를 대상으로 하는 것이 아닌, 예측해야하는 시간대의 target이 비슷한 데이터를 바탕으로
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)
- 모델을 제작

# 1. 라이브러리 로드 및 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit, KFold, StratifiedKFold

import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
import time
import datetime
import os

from scipy.sparse import csr_matrix

from mypackage import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 400)

### <div style="color:red">파일명 체크!!!</div>
- all_df: 기초작업 수준. 
- all_df4: lag 데이터 엄청 많이 만들어 놓은 것.
- all_df5: 'id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI', 'is_event', 'day', 'week', 'sell_price', 'revenue', sales_rolling_mean_t7', 'rolling_mean_t7', 'rolling_std_t7', 'sales_rolling_mean_t28', 'rolling_mean_t28', 'rolling_std_t28', 'sales_rolling_mean_t56', 'rolling_mean_t56', 'rolling_std_t56', 'sales_rolling_mean_t112','rolling_mean_t112', 'rolling_std_t112', 'sales_rolling_mean_t168', 'rolling_mean_t168', 'rolling_std_t168', 'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31', 'lag_t32', 'lag_t33', 'lag_t34', 'revenue_lag_t28', 'revenue_lag_t29', 'revenue_lag_t30','revenue_lag_t31', 'revenue_lag_t32', 'revenue_lag_t33', 'revenue_lag_t34'

In [2]:
print('data loading')
with open('inputs/all_df_val2.pickle', 'rb') as f:
    all_df = pickle.load(f)
print('data loaded')

data loading
data loaded


In [ ]:
for i in range(33, 43):
    all_df[f'lag_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(i))

In [3]:
all_df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 'event_name_1',
       'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI', 'day', 'week',
       'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean',
       'price_norm', 'price_nunique', 'item_nunique', 'price_momentum',
       'price_momentum_m', 'price_momentum_y', 'enc_cat_id_mean',
       'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std',
       'enc_state_id_cat_id_mean', 'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31',
       'lag_t32', 'lag_t33', 'lag_t34', 'lag_t35', 'lag_t36', 'lag_t37',
       'lag_t38', 'lag_t39', 'lag_t40', 'lag_t41', 'lag_t42',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t28',
       'rolling_std_t28', 'rolling_mean_t56', 'rolling_std_t56',
       'rolling_mean_t84', 'rolling_std_t84', 'rolling_mean_t112',
       'rolling_std_t112', 'rolling_mean_t168', 'rolling_std_t168',
       'price_change_

In [4]:
all_df['weekofmonth'] = np.ceil(all_df['day'] // 7).astype('int8')

In [5]:
# ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']

In [6]:
all_df['weekday'].unique()

array([2, 3, 1, 5, 6, 4, 0], dtype=int64)

In [7]:
all_df['tm_w_end'] = ((all_df['weekday'] == 1) | (all_df['weekday'] == 5))

In [8]:
all_df['tm_w_end'] = all_df['tm_w_end'].astype(np.int8)

In [ ]:
all_df['tm_w_end'] = (all_df['weekday']>=5).astype(np.int8)
all_df['tm_w_end'] = (all_df['weekday']>=5).astype(np.int8)

In [9]:
all_df.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 'event_name_1',
       'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI', 'day', 'week',
       'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean',
       'price_norm', 'price_nunique', 'item_nunique', 'price_momentum',
       'price_momentum_m', 'price_momentum_y', 'enc_cat_id_mean',
       'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std',
       'enc_state_id_cat_id_mean', 'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31',
       'lag_t32', 'lag_t33', 'lag_t34', 'lag_t35', 'lag_t36', 'lag_t37',
       'lag_t38', 'lag_t39', 'lag_t40', 'lag_t41', 'lag_t42',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t28',
       'rolling_std_t28', 'rolling_mean_t56', 'rolling_std_t56',
       'rolling_mean_t84', 'rolling_std_t84', 'rolling_mean_t112',
       'rolling_std_t112', 'rolling_mean_t168', 'rolling_std_t168',
       'price_change_

In [10]:
# grid_df['rolling_mean_t7'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(7).mean())
# grid_df['rolling_std_t7'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(7).std())

# 새롭게 만들거
# grid_df['rolling_mean_t32'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(32).mean())
# grid_df['rolling_std_t32'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(32).std())

# grid_df['rolling_mean_t64'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(64).mean())
# grid_df['rolling_std_t64'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(64).std())

# grid_df['rolling_mean_t96'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(96).mean())
# grid_df['rolling_std_t96'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(96).std())

# grid_df['rolling_mean_t180'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(180).mean())
# grid_df['rolling_std_t180'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(180).std())


# grid_df['rolling_mean_t30'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(30).mean())
# grid_df['rolling_std_t30'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(30).std())


# grid_df['rolling_mean_t90'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(90).mean())
# grid_df['rolling_std_t90'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(90).std())

# grid_df['rolling_mean_t120'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(120).mean())
# grid_df['rolling_std_t120'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(120).std())

# grid_df['rolling_mean_t180'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(180).mean())
# grid_df['rolling_std_t180'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(180).std())


# 2. 데이터 탐색

여기서는 test셋과 비슷한 시간대를 찾아서, 이 데이터만을 바탕으로 모델을 만들것임.  

test: 2016-04-25 ~ 2016-05-22  17~20주차 (16주차나 21주차까지 늘려야될지도 생각해보자)

---
train: 2015-04-27 ~ 2015-05-24  
train: 2014-04-28 ~ 2014-05-25  / 2014-04-21 ~ 2014-05-18
train: 2013-04-27 ~ 2013-05-24  
train: 2012-04-27 ~ 2012-05-24 
train: 2011-04-27 ~ 2011-05-24 

In [11]:
len(all_df)

59181090

In [12]:
all_df.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,day,week,sell_price,price_max,price_min,price_std,price_mean,price_norm,price_nunique,item_nunique,price_momentum,price_momentum_m,price_momentum_y,enc_cat_id_mean,enc_cat_id_std,enc_dept_id_mean,enc_dept_id_std,enc_state_id_cat_id_mean,lag_t28,lag_t29,lag_t30,lag_t31,lag_t32,lag_t33,lag_t34,lag_t35,lag_t36,lag_t37,lag_t38,lag_t39,lag_t40,lag_t41,lag_t42,rolling_mean_t7,rolling_std_t7,rolling_mean_t28,rolling_std_t28,rolling_mean_t56,rolling_std_t56,rolling_mean_t84,rolling_std_t84,rolling_mean_t112,rolling_std_t112,rolling_mean_t168,rolling_std_t168,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t28,weekofmonth,tm_w_end
59181085,FOODS_3_823_WI_3_validation,1432,2,0,9,2,1047,0,2016-05-22,11617,3,5,2016,30,4,0,0,0,22,20,2.980469,2.980469,2.480469,0.171631,2.800781,1.000000,5.0,206.0,1.0,1.029297,1.022461,1.616211,5.117188,2.027344,6.28125,1.545898,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.285645,0.488037,0.214233,0.567871,0.232178,0.571777,0.404785,0.823242,0.508789,1.013672,0.613281,0.990723,0.0,0.0,0.000000e+00,0.000000e+00,3,0
59181086,FOODS_3_824_WI_3_validation,1433,2,0,9,2,1047,0,2016-05-22,11617,3,5,2016,30,4,0,0,0,22,20,2.480469,2.679688,2.000000,0.253174,2.507812,0.925293,4.0,135.0,1.0,0.997070,1.112305,1.616211,5.117188,2.027344,6.28125,1.545898,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.142822,0.377930,0.321533,0.547852,0.160767,0.416748,0.107117,0.347656,0.080383,0.304199,0.053558,0.250977,0.0,0.0,5.960464e-08,0.000000e+00,3,0
59181087,FOODS_3_825_WI_3_validation,1434,2,0,9,2,1047,0,2016-05-22,11617,3,5,2016,30,4,0,0,0,22,20,3.980469,4.378906,3.980469,0.188599,4.117188,0.908691,3.0,150.0,1.0,0.965820,1.000000,1.616211,5.117188,2.027344,6.28125,1.545898,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,4.0,2.0,0.571289,0.786621,0.893066,1.166016,1.071289,1.059570,0.928711,0.979004,0.893066,0.998535,0.797852,0.951172,0.0,0.0,0.000000e+00,5.960464e-08,3,0
59181088,FOODS_3_826_WI_3_validation,1435,2,0,9,2,1047,0,2016-05-22,11617,3,5,2016,30,4,0,0,0,22,20,1.280273,1.280273,1.280273,0.000000,1.280273,1.000000,1.0,44.0,1.0,1.000000,1.000000,1.616211,5.117188,2.027344,6.28125,1.545898,3.0,1.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0,1.142578,1.344727,0.928711,1.051758,1.088867,1.066406,1.023438,1.097656,1.161133,1.326172,1.315430,1.567383,0.0,0.0,0.000000e+00,0.000000e+00,3,0
59181089,FOODS_3_827_WI_3_validation,1436,2,0,9,2,1047,0,2016-05-22,11617,3,5,2016,30,4,0,0,0,22,20,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.0,142.0,1.0,1.000000,1.000000,1.616211,5.117188,2.027344,6.28125,1.545898,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.071289,1.719727,1.517578,1.778320,1.536133,1.839844,1.588867,1.876953,1.494141,1.771484,0.0,0.0,0.000000e+00,0.000000e+00,3,0


In [13]:
c1 = all_df['week'] == 13
c2 = all_df['week'] == 14
c3 = all_df['week'] == 15
c4 = all_df['week'] == 16

c5 = all_df['week'] == 17
c6 = all_df['week'] == 18
c7 = all_df['week'] == 19
c8 = all_df['week'] == 20

all_df2 = all_df[c1 | c2 | c3 | c4 | c5 | c6 | c7 | c8]
# all_df2 = all_df[c1 | c2 | c3 | c4 | c5 | c6 | c7 | c8 |c00|c01|c02|c03]

In [14]:
len(all_df2)

10244640

In [ ]:
item_id_gb_sell_price_mean = all_df2.groupby(['id', 'year', 'week'])['sell_price'].mean().rename('item_id_gb_sell_price_mean')
all_df2 = pd.merge(all_df2, item_id_gb_sell_price_mean, on=['id', 'year', 'week'], how='left')


item_id_gb_sell_price_std = all_df2.groupby(['id', 'year', 'week'])['sell_price'].std().rename('item_id_gb_sell_price_std')
all_df2 = pd.merge(all_df2, item_id_gb_sell_price_std, on=['id', 'year', 'week'], how='left')

In [15]:
from sklearn.preprocessing import LabelEncoder

le_id = LabelEncoder()
le_id.fit(all_df2['id'])

all_df2['id'] = le_id.transform(all_df2['id'])

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
all_df2.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 'event_name_1',
       'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI', 'day', 'week',
       'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean',
       'price_norm', 'price_nunique', 'item_nunique', 'price_momentum',
       'price_momentum_m', 'price_momentum_y', 'enc_cat_id_mean',
       'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std',
       'enc_state_id_cat_id_mean', 'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31',
       'lag_t32', 'lag_t33', 'lag_t34', 'lag_t35', 'lag_t36', 'lag_t37',
       'lag_t38', 'lag_t39', 'lag_t40', 'lag_t41', 'lag_t42',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t28',
       'rolling_std_t28', 'rolling_mean_t56', 'rolling_std_t56',
       'rolling_mean_t84', 'rolling_std_t84', 'rolling_mean_t112',
       'rolling_std_t112', 'rolling_mean_t168', 'rolling_std_t168',
       'price_change_

# 3. 피쳐 엔지니어링

## 3.1 train, test

prepare training and test data.
- 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

# 4. 모델 제작

## 4.1 train set /  test set 선정

feature 선정에 있어서 다른 노트북에서는 all_df = all_df.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1) 이렇게 진행했음

In [17]:
# all_df2 = all_df2.rename(columns={'target':'sales'})

2011-01-29 ~ 2016-04-24 : d_1 ~ d_1913  
2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)  
2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)  

In [18]:
all_df2.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 'event_name_1',
       'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI', 'day', 'week',
       'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean',
       'price_norm', 'price_nunique', 'item_nunique', 'price_momentum',
       'price_momentum_m', 'price_momentum_y', 'enc_cat_id_mean',
       'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std',
       'enc_state_id_cat_id_mean', 'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31',
       'lag_t32', 'lag_t33', 'lag_t34', 'lag_t35', 'lag_t36', 'lag_t37',
       'lag_t38', 'lag_t39', 'lag_t40', 'lag_t41', 'lag_t42',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t28',
       'rolling_std_t28', 'rolling_mean_t56', 'rolling_std_t56',
       'rolling_mean_t84', 'rolling_std_t84', 'rolling_mean_t112',
       'rolling_std_t112', 'rolling_mean_t168', 'rolling_std_t168',
       'price_change_

In [19]:
DROP_COLS = ['sales', 'date', 'wm_yr_wk']

In [20]:
train_set_X = all_df2[all_df2['date'] <= '2016-04-24']
# train_weights = train_set['weight'].copy()
# train_set_X = train_set[features]
train_set_y = train_set_X['sales']
train_set_X = train_set_X.drop(DROP_COLS, axis=1)

# 테스트 셋
test = all_df2[all_df2['date'] > '2016-04-24']
test_set = test.drop(DROP_COLS, axis=1)


var_set_X = all_df2[(all_df2['date'] > '2015-04-24') & (all_df2['date'] <= '2015-05-22')]
var_set_y = var_set_X['sales']
var_set_X = var_set_X.drop(DROP_COLS, axis=1)

In [21]:
StratifiedKFold

sklearn.model_selection._split.StratifiedKFold

In [22]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=30)
# folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=30)

splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'regression',
        boosting_type = 'gbdt',
        num_leaves = 2048,
        colsample_bytree = 0.8,
        subsample = 0.5,
        n_estimators = 1200, ## 중요!!!!
        learning_rate = 0.03,
        n_jobs = -1,
        reg_lambda = 0.1
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 10, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds



sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])




Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'weekday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA',
       'snap_TX', 'snap_WI', 'day', 'week', 'sell_price', 'price_max',
       'price_min', 'price_std', 'price_mean', 'price_norm', 'price_nunique',
       'item_nunique', 'price_momentum', 'price_momentum_m',
       'price_momentum_y', 'enc_cat_id_mean', 'enc_cat_id_std',
       'enc_dept_id_mean', 'enc_dept_id_std', 'enc_state_id_cat_id_mean',
       'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31', 'lag_t32', 'lag_t33',
       'lag_t34', 'lag_t35', 'lag_t36', 'lag_t37', 'lag_t38', 'lag_t39',
       'lag_t40', 'lag_t41', 'lag_t42', 'rolling_mean_t7', 'rolling_std_t7',
       'rolling_mean_t28', 'rolling_std_t28', 'rolling_mean_t56',
       'rolling_std_t56', 'rolling_mean_t84', 'rolling_std_t84',
       'rolling_mean_t112', 'rolling_std_t112', 'rolling_mean_t168',
       'rolling_std_t168', 'price_change_t1', 'price_change_t365',
       'ro

[257]	valid_0's l2: 3.90406
[258]	valid_0's l2: 3.90245
[259]	valid_0's l2: 3.90139
[260]	valid_0's l2: 3.90092
[261]	valid_0's l2: 3.90045
[262]	valid_0's l2: 3.90002
[263]	valid_0's l2: 3.89981
[264]	valid_0's l2: 3.89722
[265]	valid_0's l2: 3.89679
[266]	valid_0's l2: 3.89455
[267]	valid_0's l2: 3.89447
[268]	valid_0's l2: 3.89376
[269]	valid_0's l2: 3.89352
[270]	valid_0's l2: 3.89144
[271]	valid_0's l2: 3.8913
[272]	valid_0's l2: 3.88885
[273]	valid_0's l2: 3.88867
[274]	valid_0's l2: 3.88698
[275]	valid_0's l2: 3.88627
[276]	valid_0's l2: 3.88572
[277]	valid_0's l2: 3.88522
[278]	valid_0's l2: 3.88468
[279]	valid_0's l2: 3.88422
[280]	valid_0's l2: 3.88277
[281]	valid_0's l2: 3.88241
[282]	valid_0's l2: 3.88157
[283]	valid_0's l2: 3.88122
[284]	valid_0's l2: 3.88093
[285]	valid_0's l2: 3.87975
[286]	valid_0's l2: 3.87787
[287]	valid_0's l2: 3.87699
[288]	valid_0's l2: 3.87646
[289]	valid_0's l2: 3.8754
[290]	valid_0's l2: 3.87491
[291]	valid_0's l2: 3.87438
[292]	valid_0's l2: 3.

[551]	valid_0's l2: 3.77736
[552]	valid_0's l2: 3.77685
[553]	valid_0's l2: 3.77685
[554]	valid_0's l2: 3.77654
[555]	valid_0's l2: 3.77597
[556]	valid_0's l2: 3.77587
[557]	valid_0's l2: 3.77552
[558]	valid_0's l2: 3.77532
[559]	valid_0's l2: 3.775
[560]	valid_0's l2: 3.77493
[561]	valid_0's l2: 3.77474
[562]	valid_0's l2: 3.77474
[563]	valid_0's l2: 3.77446
[564]	valid_0's l2: 3.7743
[565]	valid_0's l2: 3.77429
[566]	valid_0's l2: 3.77432
[567]	valid_0's l2: 3.77421
[568]	valid_0's l2: 3.77412
[569]	valid_0's l2: 3.77403
[570]	valid_0's l2: 3.77419
[571]	valid_0's l2: 3.77408
[572]	valid_0's l2: 3.77402
[573]	valid_0's l2: 3.77361
[574]	valid_0's l2: 3.77318
[575]	valid_0's l2: 3.7732
[576]	valid_0's l2: 3.77303
[577]	valid_0's l2: 3.77296
[578]	valid_0's l2: 3.77266
[579]	valid_0's l2: 3.77248
[580]	valid_0's l2: 3.77243
[581]	valid_0's l2: 3.77235
[582]	valid_0's l2: 3.77225
[583]	valid_0's l2: 3.77197
[584]	valid_0's l2: 3.77202
[585]	valid_0's l2: 3.77204
[586]	valid_0's l2: 3.77

[845]	valid_0's l2: 3.73629
[846]	valid_0's l2: 3.736
[847]	valid_0's l2: 3.73596
[848]	valid_0's l2: 3.73588
[849]	valid_0's l2: 3.73583
[850]	valid_0's l2: 3.73538
[851]	valid_0's l2: 3.73529
[852]	valid_0's l2: 3.73527
[853]	valid_0's l2: 3.73519
[854]	valid_0's l2: 3.73496
[855]	valid_0's l2: 3.73494
[856]	valid_0's l2: 3.73485
[857]	valid_0's l2: 3.73454
[858]	valid_0's l2: 3.73451
[859]	valid_0's l2: 3.73434
[860]	valid_0's l2: 3.73438
[861]	valid_0's l2: 3.7344
[862]	valid_0's l2: 3.73436
[863]	valid_0's l2: 3.73422
[864]	valid_0's l2: 3.73422
[865]	valid_0's l2: 3.73405
[866]	valid_0's l2: 3.73378
[867]	valid_0's l2: 3.73388
[868]	valid_0's l2: 3.7338
[869]	valid_0's l2: 3.73373
[870]	valid_0's l2: 3.73335
[871]	valid_0's l2: 3.73315
[872]	valid_0's l2: 3.73292
[873]	valid_0's l2: 3.73276
[874]	valid_0's l2: 3.73263
[875]	valid_0's l2: 3.73266
[876]	valid_0's l2: 3.73244
[877]	valid_0's l2: 3.7325
[878]	valid_0's l2: 3.73247
[879]	valid_0's l2: 3.73225
[880]	valid_0's l2: 3.732

[1135]	valid_0's l2: 3.7127
[1136]	valid_0's l2: 3.71266
[1137]	valid_0's l2: 3.71266
[1138]	valid_0's l2: 3.71267
[1139]	valid_0's l2: 3.71264
[1140]	valid_0's l2: 3.71256
[1141]	valid_0's l2: 3.71259
[1142]	valid_0's l2: 3.71262
[1143]	valid_0's l2: 3.71259
[1144]	valid_0's l2: 3.71247
[1145]	valid_0's l2: 3.71243
[1146]	valid_0's l2: 3.71242
[1147]	valid_0's l2: 3.71241
[1148]	valid_0's l2: 3.7123
[1149]	valid_0's l2: 3.71228
[1150]	valid_0's l2: 3.71219
[1151]	valid_0's l2: 3.71207
[1152]	valid_0's l2: 3.71198
[1153]	valid_0's l2: 3.71188
[1154]	valid_0's l2: 3.71185
[1155]	valid_0's l2: 3.71187
[1156]	valid_0's l2: 3.71171
[1157]	valid_0's l2: 3.71164
[1158]	valid_0's l2: 3.71158
[1159]	valid_0's l2: 3.71127
[1160]	valid_0's l2: 3.71123
[1161]	valid_0's l2: 3.71123
[1162]	valid_0's l2: 3.71123
[1163]	valid_0's l2: 3.71109
[1164]	valid_0's l2: 3.71111
[1165]	valid_0's l2: 3.71108
[1166]	valid_0's l2: 3.71094
[1167]	valid_0's l2: 3.71093
[1168]	valid_0's l2: 3.71081
[1169]	valid_0's

...

[115]	valid_0's l2: 4.34972
[116]	valid_0's l2: 4.34434
[117]	valid_0's l2: 4.34159
[118]	valid_0's l2: 4.33712
[119]	valid_0's l2: 4.33435
[120]	valid_0's l2: 4.33163
[121]	valid_0's l2: 4.32815
[122]	valid_0's l2: 4.32467
[123]	valid_0's l2: 4.32076
[124]	valid_0's l2: 4.31737
[125]	valid_0's l2: 4.31262
[126]	valid_0's l2: 4.31061
[127]	valid_0's l2: 4.30619
[128]	valid_0's l2: 4.30395
[129]	valid_0's l2: 4.29919
[130]	valid_0's l2: 4.29458
[131]	valid_0's l2: 4.29218
[132]	valid_0's l2: 4.28999
[133]	valid_0's l2: 4.28782
[134]	valid_0's l2: 4.286
[135]	valid_0's l2: 4.28328
[136]	valid_0's l2: 4.27997
[137]	valid_0's l2: 4.2784
[138]	valid_0's l2: 4.27695
[139]	valid_0's l2: 4.27497
[140]	valid_0's l2: 4.27118
[141]	valid_0's l2: 4.26932
[142]	valid_0's l2: 4.26717
[143]	valid_0's l2: 4.26488
[144]	valid_0's l2: 4.26157
[145]	valid_0's l2: 4.25818
[146]	valid_0's l2: 4.25502
[147]	valid_0's l2: 4.25206
[148]	valid_0's l2: 4.24941
[149]	valid_0's l2: 4.24677
[150]	valid_0's l2: 4.2

[409]	valid_0's l2: 3.94638
[410]	valid_0's l2: 3.94632
[411]	valid_0's l2: 3.94615
[412]	valid_0's l2: 3.94553
[413]	valid_0's l2: 3.94548
[414]	valid_0's l2: 3.94533
[415]	valid_0's l2: 3.9452
[416]	valid_0's l2: 3.94452
[417]	valid_0's l2: 3.94402
[418]	valid_0's l2: 3.94382
[419]	valid_0's l2: 3.94383
[420]	valid_0's l2: 3.94347
[421]	valid_0's l2: 3.94357
[422]	valid_0's l2: 3.94289
[423]	valid_0's l2: 3.94144
[424]	valid_0's l2: 3.94094
[425]	valid_0's l2: 3.94076
[426]	valid_0's l2: 3.94044
[427]	valid_0's l2: 3.94003
[428]	valid_0's l2: 3.93992
[429]	valid_0's l2: 3.93937
[430]	valid_0's l2: 3.93861
[431]	valid_0's l2: 3.93843
[432]	valid_0's l2: 3.93771
[433]	valid_0's l2: 3.93757
[434]	valid_0's l2: 3.93751
[435]	valid_0's l2: 3.93681
[436]	valid_0's l2: 3.93657
[437]	valid_0's l2: 3.93633
[438]	valid_0's l2: 3.93638
[439]	valid_0's l2: 3.93633
[440]	valid_0's l2: 3.93607
[441]	valid_0's l2: 3.93566
[442]	valid_0's l2: 3.93552
[443]	valid_0's l2: 3.93545
[444]	valid_0's l2: 3

[703]	valid_0's l2: 3.8812
[704]	valid_0's l2: 3.88122
[705]	valid_0's l2: 3.88088
[706]	valid_0's l2: 3.88077
[707]	valid_0's l2: 3.8807
[708]	valid_0's l2: 3.88042
[709]	valid_0's l2: 3.88015
[710]	valid_0's l2: 3.87974
[711]	valid_0's l2: 3.87962
[712]	valid_0's l2: 3.87948
[713]	valid_0's l2: 3.87907
[714]	valid_0's l2: 3.87893
[715]	valid_0's l2: 3.87888
[716]	valid_0's l2: 3.87865
[717]	valid_0's l2: 3.87863
[718]	valid_0's l2: 3.87843
[719]	valid_0's l2: 3.87843
[720]	valid_0's l2: 3.87848
[721]	valid_0's l2: 3.87836
[722]	valid_0's l2: 3.87836
[723]	valid_0's l2: 3.87845
[724]	valid_0's l2: 3.87821
[725]	valid_0's l2: 3.87775
[726]	valid_0's l2: 3.87763
[727]	valid_0's l2: 3.87749
[728]	valid_0's l2: 3.8774
[729]	valid_0's l2: 3.87715
[730]	valid_0's l2: 3.87713
[731]	valid_0's l2: 3.87707
[732]	valid_0's l2: 3.87663
[733]	valid_0's l2: 3.87643
[734]	valid_0's l2: 3.87648
[735]	valid_0's l2: 3.87603
[736]	valid_0's l2: 3.876
[737]	valid_0's l2: 3.87599
[738]	valid_0's l2: 3.875

[997]	valid_0's l2: 3.84992
[998]	valid_0's l2: 3.84996
[999]	valid_0's l2: 3.84996
[1000]	valid_0's l2: 3.84998
[1001]	valid_0's l2: 3.84976
[1002]	valid_0's l2: 3.84974
[1003]	valid_0's l2: 3.84978
[1004]	valid_0's l2: 3.84966
[1005]	valid_0's l2: 3.84963
[1006]	valid_0's l2: 3.84964
[1007]	valid_0's l2: 3.84913
[1008]	valid_0's l2: 3.84906
[1009]	valid_0's l2: 3.84903
[1010]	valid_0's l2: 3.84899
[1011]	valid_0's l2: 3.84897
[1012]	valid_0's l2: 3.84899
[1013]	valid_0's l2: 3.84898
[1014]	valid_0's l2: 3.84898
[1015]	valid_0's l2: 3.84897
[1016]	valid_0's l2: 3.8488
[1017]	valid_0's l2: 3.84883
[1018]	valid_0's l2: 3.84882
[1019]	valid_0's l2: 3.84881
[1020]	valid_0's l2: 3.84862
[1021]	valid_0's l2: 3.84857
[1022]	valid_0's l2: 3.84844
[1023]	valid_0's l2: 3.84831
[1024]	valid_0's l2: 3.84824
[1025]	valid_0's l2: 3.84824
[1026]	valid_0's l2: 3.84819
[1027]	valid_0's l2: 3.84815
[1028]	valid_0's l2: 3.8481
[1029]	valid_0's l2: 3.84809
[1030]	valid_0's l2: 3.84806
[1031]	valid_0's l2

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [23]:
teams_now = 3215
teams_before = 3186
final['F1'] *= teams_now/teams_before

magic = final['F2'].sum()
korea_emergency = 119
final['F2'] *= magic / (magic + korea_emergency)


final['F3'] *= 1913/1923


sex_constant = 1.0073
final['F4'] /= sex_constant


color_of_the_night = 995874
color_of_the_night /= 1000000
final['F5'] *= color_of_the_night

injury = 1.000376
final['F6'] *= injury


import numpy as np
np.random.seed(198505)
correction = np.random.randint(1000000)/1000000
final['F7'] *= correction


final['F8'] *= (100-1)/100 + (100-12)/10000

final['F11'] = final['F11']

for i in range(9,20):
    if i!=11:
        final['F'+str(i)] *= 1.01 
        print(final['F'+str(i)].sum())

for i in range(20,29):
    final['F'+str(i)] *= 1.02
    print(final['F'+str(i)].sum())

42149.70869323304
38461.08722861133
45824.61123149472
53793.10801155053
46021.851090389944
43724.02961338125
38891.06634097094
39724.47515891408
39880.709823487625
43534.83766431842
52849.78458765072
54743.25421310454
40923.503833968745
37954.114134627096
37053.96907149357
37233.80256593792
41821.20542812752
50721.29121926315
51831.01125453153


In [24]:
final.to_csv('submissions/submission_lgb3.csv', index = False)

In [ ]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=30)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns

val1 = []
val2 = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    cb = CatBoostRegressor(n_estimators = 4000, task_type='GPU')
    cb.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=10, use_best_model=True)

    feature_importances[f'fold_{fold_n + 1}'] = cb.feature_importances_
    
    y_pred_valid = cb.predict(X_valid)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    val1.append(val_score)
    
    all_pred = cb.predict(var_set_X)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    val2.append(all_var_score)
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += cb.predict(test_set) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = cb.get_params()
# eval_results = cb.get_evals_result()['validation']['RMSE']

print('mean rmse score over folds is',np.mean(val1))
test['sales'] = y_preds



sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])




In [ ]:
np.mean(val1)

In [ ]:
np.mean(val1)

In [ ]:
np.mean(val2)

In [ ]:
for i in range(1,29):
    final['F'+str(i)] *= 1.0315
final.to_csv('submissions/submission_cb.csv', index = False)

# time.sleep(2)
# os.chdir("submissions")
# !kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
# os.chdir("../")


In [ ]:
sub1 = pd.read_csv('submissions/submission_lgb.csv')
sub2 = pd.read_csv('submissions/submission_cb.csv')



In [ ]:
for i in range(1, 29):
    sub1.iloc[:, i] = sub1.iloc[:, i] * 0.8 + sub2.iloc[:, i] * 0.2

In [ ]:
sub1.to_csv('sub_ensemble.csv', index = False)

In [ ]:
def split_store(data, store_id):
    data = data[data["store_id"] == store_id]
    return data

In [ ]:
def make_train_test_set(data, drop_cols):

    features = data.columns.drop(drop_cols)

    train_set = data[data['date'] <= '2016-04-24']
    train_set_X = train_set[features]
    train_set_y = train_set['target']

    # 테스트 셋
    test = data[data['date'] > '2016-04-24']
    test_set = test[features]

    var_set = data[(data['date'] > '2015-04-27') & (data['date'] <= '2015-05-22')]
    var_set_X = var_set[features]
    var_set_y = var_set['target']
    
    return train_set_X, train_set_y, test, test_set, var_set_X, var_set_y
    
    

In [ ]:
drop_cols = ['id', 'd', 'target', 'date', 'wm_yr_wk', 'is_event', 'lag_t28', 'lag_t29',
       'lag_t30', 'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27']

train_set_X, train_set_y, test, test_set, var_set_X, var_set_y = make_train_test_set(all_df2, drop_cols)

In [ ]:
all_df2[all_df2.store_id == 8]

## 4.2 KFold - LGBM 모델

In [ ]:
def lgb_model(data, drop_cols):
    train_set_X, train_set_y, test, test_set, var_set_X, var_set_y = make_train_test_set(data, drop_cols)
    n_fold = 5
    folds = KFold(n_splits=n_fold, shuffle=True, random_state=333)
    splits = folds.split(train_set_X, train_set_y)

    y_preds = np.zeros(test_set.shape[0])
    y_oof = np.zeros(train_set_X.shape[0])

    feature_importances = pd.DataFrame()
    feature_importances['feature'] = train_set_X.columns
    mean_score = []


    for fold_n, (train_index, valid_index) in enumerate(splits):
        print('Fold:',fold_n+1)

        X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
        y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]

        lgb = LGBMRegressor(
            boosting_type = 'gbdt',
            num_leaves = 1024,
            colsample_bytree = 0.8,
            subsample = 0.8,
            n_estimators = 300, ## 중요!!!!
            learning_rate = 0.1,
            n_jobs = -1,
            device = 'gpu',
            random_state=333
        )
        lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

        feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_

        y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)

        y_oof[valid_index] = y_pred_valid

        # validation 측정
        val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
        print(f'val rmse score is {val_score}')
        mean_score.append(val_score)

        all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
        all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
        print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')


        # 예측
        y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold

        # 메모리 정리
        del X_train, X_valid, y_train, y_valid

    features = train_set_X.columns
    params = lgb.get_params()
    eval_results = lgb.evals_result_['valid_0']['l2']

    print('mean rmse score over folds is',np.mean(mean_score))
    test['sales'] = y_preds
    write_params_features(features, params, eval_results)
    save_feature_importance(feature_importances)
    
    return test

In [ ]:
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

In [ ]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'tweedie',
        tweedie_variance_power= 1.1,
#         objective = 'poisson',
        metric = "rmse",
        boosting_type = 'gbdt',
        num_leaves = 128,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 1200, ## 중요!!!!
        learning_rate = 0.05,
        n_jobs = -1,
        reg_lambda = 1,
#         device = 'gpu'
    )

    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
            early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds



In [ ]:
test['sales'] = y_preds

In [ ]:
feature_importances

In [ ]:
mean_score

## 4.5 베이지안 최적화(Bayesian optimization
:  초매개변수(Hyper parameter)를 자동으로 조정

In [ ]:
from bayes_opt import BayesianOptimization
from functools import partial   

In [ ]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    kf = KFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = LGBMRegressor(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
            device = 'gpu'
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict(x_valid)
        true = y_valid
        score += metrics.mean_squared_error(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
# 모델과 관련없는 변수 고정
func_fixed = partial(lgb_cv, x_data=train_set_X.reset_index(drop=True), y_data=train_set_y.reset_index(drop=True), n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (64, 4096),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0.4, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0.4, 1),      # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
lgbBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!

In [ ]:
feature_importances

## 4.3 store 별로 모델 만들기

In [ ]:
drop_cols = ['id', 'd', 'target', 'date', 'wm_yr_wk', 'is_event', 'lag_t28', 'lag_t29',
       'lag_t30', 'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27']

test_sets = []
for i in range(10):
    store_i = split_store(all_df2, i)
    test_sets.append(lgb_model(store_i, drop_cols))

In [ ]:
for i in range(1, 10):
    test_sets[0] = pd.concat([test_sets[0], test_sets[i]])

In [ ]:
test = test_sets[0].sort_index()

## 4.4 keras 모델

In [ ]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# model.compile(optimizer = "rmsprop", loss = root_mean_squared_error, 
#               metrics =["accuracy"])

In [ ]:
from keras.models import Sequential             # Sequential 생성자를 불러옵니다.
from keras.layers import Dense, Activation, Dropout      # Dense와 Activation 두 층 인스턴스를 불러옵니다.

model = Sequential()
model.add(Dense(128, activation='relu', kernel_initializer='he_normal', input_dim=train_set_X[features].shape[1]))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
# model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='linear'))                   # 1차원 출력에 'sigmoid' 함수를 적용하는 Dense 층입니다.
model.compile(optimizer='rmsprop',                          # 최적화 함수 = 'rmsprop'
              loss= 'mse',                   # 손실 함수 = 'binary_crossentropy'
              metrics=[root_mean_squared_error])                         # 평가 지표 = 'accuracy'

# 학습시키기 
model.fit(train_set_X, train_set_y, epochs=1, batch_size=1000, validation_split = 0.2)           # 생성된 데이터를 32개씩의 배치로 나누어 전체를 총 10회 학습시킵니다.

In [ ]:
test_set.head()

In [ ]:
result_ = model.predict(test_set)

In [ ]:
result_

In [ ]:
test['target'] = result_

# 5. 예측 및 제출

In [ ]:
sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])

for i in range(1,29):
    final['F'+str(i)] *= 1.0315
final.to_csv('submissions/submission.csv', index = False)

In [ ]:
len(final)

In [ ]:
final.head()

In [ ]:
# 새롭게 구한 값도 보정하니 정확도가 높아짐
# 1.04 를 곱하라고 했는데 1.03 곱하니 더 정확도가 높아짐
# 이차회귀식으로 계산 결과, 1.0315 곱하는게 가장 좋음.
# tmp = pd.read_csv('submissions/sub_dt_lgb.csv')
# for i in range(1,29):
#     tmp['F'+str(i)] *= 1.0315
# tmp.to_csv('submissions/submission.csv', index = False)

https://www.kaggle.com/c/m5-forecasting-accuracy/submit

In [ ]:
time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")

# 6. 모델 파라미터 및 피처 기록 및 모델 저장하기

In [ ]:
write_params_features(features, params, eval_results, mean_score)

In [ ]:
save_feature_importance(feature_importances)

In [ ]:
## 1등 노트북이랑 비교해서 카테고리형 피처 빠진거있나 확인하기

In [ ]:
drop_cols = ['d', 'sales', 'date', 'wm_yr_wk', 'is_event', 'revenue',
             'lag_t1', 'lag_t2', 'lag_t3', 'lag_t4',
       'lag_t5', 'lag_t6', 'lag_t7', 'lag_t8', 'lag_t9', 'lag_t10', 'lag_t11',
       'lag_t12', 'lag_t13', 'lag_t14', 'lag_t15', 'lag_t16', 'lag_t17',
       'lag_t18', 'lag_t19', 'lag_t20', 'lag_t21', 'lag_t22', 'lag_t23',
       'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27', 'lag_t32', 'lag_t33', 'lag_t34',
       'rolling_max_s7_t7', 'rolling_min_s7_t7', 
       'rolling_std_s7_t7', 'rolling_max_s7_t28', 'rolling_min_s7_t28',
        'rolling_std_s7_t28', 'rolling_max_s7_t112',
       'rolling_min_s7_t112', 'rolling_mean_s7_t112', 'rolling_std_s7_t112',
       'rolling_max_s7_t168', 'rolling_min_s7_t168', 'rolling_mean_s7_t168',
       'rolling_std_s7_t168',  'rolling_max_s28_t7', 'rolling_min_s28_t7',  'rolling_max_s28_t28',
       'rolling_min_s28_t28',   'rolling_max_s28_t112', 'rolling_min_s28_t112',  'rolling_std_s28_t112',
          'rolling_max_s28_t168', 'rolling_min_s28_t168',    'rolling_std_s28_t168']

features = all_df2.columns.drop(drop_cols)

In [ ]:
all_df2.loc[3759341, 'sales'] = 200

train_set = all_df2[all_df2['date'] <= '2016-04-24']
train_set_X = train_set[features]
train_set_y = train_set['sales']

# 테스트 셋
test = all_df2[all_df2['date'] > '2016-04-24']
test_set = test[features]


var_set = all_df2[(all_df2['date'] > '2015-04-27') & (all_df2['date'] <= '2015-05-22')]
var_set_X = var_set[features]
var_set_y = var_set['sales']

In [ ]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'regression',
        boosting_type = 'gbdt',
        num_leaves = 2048,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 1200, ## 중요!!!!
        learning_rate = 0.05,
        n_jobs = -1,
        reg_lambda = 1,
        device = 'gpu'
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds



In [ ]:
import pandas as pd

tmp1 = pd.read_csv('reference-val/submission_v1_val_correction.csv')
tmp2 = pd.read_csv('submissions/submission_lgb_correction.csv')



In [ ]:
for i in range(1, 29):
    tmp1.iloc[:, i] = tmp1.iloc[:, i] * 0.9 + tmp2.iloc[:, i] * 0.1

In [ ]:
tmp1.to_csv('ens_tmp.csv', index=False)